# Import libraries

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt

# Data exploration

## Load data

In [2]:
# Load data
x_train = pd.read_csv("../raw_data/X_train_v2.csv", index_col=0)
x_test = pd.read_csv("../raw_data/X_test_v2.csv", index_col=0)
y_train = pd.read_csv("../raw_data/Y_train_sl9m6Jh.csv", index_col=0)

# Concatenate x_rtain and x_test for exploration
df = pd.concat([x_train, x_test])

# Define Time column as datetime
df.loc[:, 'Time'] = pd.to_datetime(df['Time'], format='%d/%m/%Y %H:%M')
df.loc[:, 'WF'] = df["WF"].apply(lambda x: int(x.strip('WF')))

# Add y_train in a column named Production
df.loc[y_train.index, 'Production'] = y_train

In [3]:
df.loc[51:54]

,WF,Time,NWP1_00h_D-2_U,NWP1_00h_D-2_V,NWP1_00h_D-2_T,NWP1_06h_D-2_U,NWP1_06h_D-2_V,NWP1_06h_D-2_T,NWP1_12h_D-2_U,NWP1_12h_D-2_V,...,NWP4_12h_D-1_U,NWP4_12h_D-1_V,NWP4_12h_D-1_CLCT,NWP4_00h_D_U,NWP4_00h_D_V,NWP4_00h_D_CLCT,NWP4_12h_D_U,NWP4_12h_D_V,NWP4_12h_D_CLCT,Production
ID,,,,,,,,,,,,,,,,,,,,,
51,1,2018-05-03 03:00:00,4.4229,-16.527,288.68,6.3428,-16.317,288.57,2.8521,-15.495,...,4.295196,-7.850803,-0.000016,4.450514,-6.487014,-0.000013,NaN,NaN,NaN,5.39
52,1,2018-05-03 04:00:00,5.0740,-16.985,288.40,6.7567,-16.895,288.23,2.7426,-16.285,...,4.295196,-8.693379,-0.000016,4.369153,-7.600720,-0.000013,NaN,NaN,NaN,5.11
53,1,2018-05-03 05:00:00,5.8899,-17.983,288.21,6.8435,-17.223,287.99,3.0243,-17.227,...,4.292873,-8.995411,-0.000016,3.888633,-8.621414,-0.000013,NaN,NaN,NaN,5.03
54,1,2018-05-03 06:00:00,6.1131,-18.897,287.97,7.1638,-17.580,287.95,3.4257,-17.655,...,4.220859,-9.930776,-0.000016,3.100145,-9.882844,0.035084,NaN,NaN,NaN,7.18


_Meteorological variables_: Numerical Weather Predictions are provided by meteorological centers several times a day (updates), typically at 00h UTC, 06h UTC, 12h UTC and 18h UTC. We call these sets of forecasts "Runs". Consequently, if the input file contains forecasts arising from several runs, this implies that a single NWP is associated with several forecasts for the same forecasting time

The format of the header of the csv files for the meteorological variables is the following:
*NWPi_HourOfTheRun_DayOfTheRun_Variable*

With:
- _NWPi_ the considered Numerical Weather Prediction model (meteorological model);
- _HourOfTheRun_ the hour (UTC) of the considered run. According to the NWP, it could be 00h, 06h, 12h and 18h (case of NWP with 4 runs per day) or only 00h and 12h (case of NWP with 2 runs per day);
- _DayOfTheRun_ the day of the considered run. We provide in the csv files predictions from the D-2 day runs (the day before yesterday), D-1 day runs (yesterday) and D day runs;
- _Variables_ the different meteorological variables forecasted by the NWP. These are essentially U, V and T:

 - _U and V components_ of the wind at 100m (or 10m) height (m/s): these are the zonal and meridional velocities of the wind, respectively. Both are given at a height of 100m above ground for NWP1, NWP2 and NWP3. U and V are given at a height of 10m for NWP4. Even if these variables are given at hourly timestep, **the temporal representativity of the given values is for a 10-minutes window ranging from H-10 min to H**
  - _Temperature of air (°C), abbreviated T_: this is the averaged temperature over the entire hour (from H-1 to H). Wind power production is sensitive to air temperature since it affects the air density. **This variable is provided only for NWP1 and NWP3**
  - _Total cloud cover (%), abbreviated CLCT_: this is the total cloud cover of the sky, ranging from 0% (clear sky, no cloud) to 100% (fully clouded sky). **The value is an instant value at hour H. This variable is provided only for NWP4.**

NWP Variable | Timeframe | NWP 1 (hourly) | NWP 2 (every 3 hours) | NWP 3 (every 3 hours) | NWP 4 (hourly)
------ | ----- | ----- | ----- | ----- | ----- 
Wind speed U,V (m/s) | 10 min average [H-10min,H] | x (@100m) | x (@100m) | x (@100m) | x (@10m) 
Temperature of air T (m/s) | 1 hour average [H-1,H] | x | | x | 
Total cloud cover CLCT (%) | instant value at H | | | | x

In [4]:
# Number of periods par Wind Farm in training and test datasets respectively
df.assign(
    training = df.Production.isna(), 
    test = ~df.Production.isna())[['WF','training','test']].groupby("WF").sum()

,training,test
WF,,
1,6190,6239
2,6190,6239
3,6190,6239
4,6190,6239
5,5579,6180
6,6190,6239


## Preprocessing

### Calculate best forecasts

For a given target time, a NWP model is forecasting a weather variable several times (at different delays before the target time).

In [6]:
df.loc[51:56,['WF','Time']+[col for col in df.columns if col.startswith('NWP4') and col.endswith('_U')]]

,WF,Time,NWP4_00h_D-2_U,NWP4_12h_D-2_U,NWP4_00h_D-1_U,NWP4_12h_D-1_U,NWP4_00h_D_U,NWP4_12h_D_U
ID,,,,,,,,
51,1,2018-05-03 03:00:00,NaN,5.144098,4.668093,4.295196,4.450514,NaN
52,1,2018-05-03 04:00:00,NaN,4.850463,4.863529,4.295196,4.369153,NaN
53,1,2018-05-03 05:00:00,NaN,4.834910,4.863529,4.292873,3.888633,NaN
54,1,2018-05-03 06:00:00,NaN,4.677517,4.659237,4.220859,3.100145,NaN
55,1,2018-05-03 07:00:00,NaN,4.286832,4.404757,3.591898,2.557376,NaN
56,1,2018-05-03 08:00:00,NaN,3.910456,4.382320,3.244605,1.746500,NaN


Best forecasts by NWP will be computed using a weighted mean of the forecasts with a memory coefficient as hyperparameter

In [7]:
def get_run_infos(run_name):
    """ Returns predictor name and timedelta from run name"""
    predictor = '_'.join(run_name.split('_')[::3])
    run_day_offset_str = run_name.split('_')[2].strip('D')
    run_day_offset = int(run_day_offset_str) if run_day_offset_str != '' else 0
    run_time = int(run_name.split('_')[1].strip('h'))
    timedelta = dt.timedelta(days=run_day_offset, hours=run_time)
    return predictor, timedelta

In [8]:
def reshape_run_data(run_data):
    """ Returns run data in a dataframe with predictor name, delay and value columns """
    # Preparation
    run_name = [col for col in run_data.columns if col.startswith('NWP')][0]

    # Computing predictor and delay
    predictor, timedelta = get_run_infos(run_name)
    run_data.loc[:, 'predictor'] = predictor
    update_time = run_data.Time.dt.normalize() + timedelta
    run_data.loc[:, 'delay'] = (run_data.Time - update_time) / pd.Timedelta('1h')
    # Cleaning
    run_data.rename(columns={run_name: 'value'}, inplace=True)
    run_data.drop(columns='Time', inplace=True)
    run_data.dropna(subset=['value'], inplace=True)
    run_data.reset_index(inplace=True)
    return run_data

In [9]:
# Reshape and concatenate each NWP_run data ( index=[ID,predictor]] , columns=[delay,value] )
best_forecasts = pd.concat([reshape_run_data(df.loc[:, ['Time', col]])
                            for col in df.columns if col.startswith('NWP')], ignore_index=True)

In [10]:
best_forecasts.head()

,ID,value,predictor,delay
0,48,5.4041,NWP1_U,48.0
1,49,4.8424,NWP1_U,49.0
2,50,4.4050,NWP1_U,50.0
3,51,4.4229,NWP1_U,51.0
4,52,5.0740,NWP1_U,52.0


In [11]:
# Computing weight and weighted_value
forecast_memory = 0.2
best_forecasts.loc[:, 'weight'] = forecast_memory ** best_forecasts.delay
best_forecasts.loc[:, 'weighted_value'] = best_forecasts.weight * best_forecasts.value
best_forecasts

# Computing best forecasts
gb = best_forecasts.groupby(['ID', 'predictor'])
best_forecasts = gb.weighted_value.sum() / gb.weight.sum()
best_forecasts = best_forecasts.unstack(level=-1)

# Replace initial NWP runs forecasts by the best computed forecasts
not_nwp_cols = [col for col in df.columns if not col.startswith('NWP')]
df_best = pd.concat([df[not_nwp_cols], best_forecasts], axis=1)
df_best.head()

,WF,Time,Production,NWP1_T,NWP1_U,NWP1_V,NWP2_U,NWP2_V,NWP3_T,NWP3_U,NWP3_V,NWP4_CLCT,NWP4_U,NWP4_V
ID,,,,,,,,,,,,,,
1,1,2018-05-01 01:00:00,0.02,286.44,-2.2485,-3.2578,NaN,NaN,NaN,NaN,NaN,82.543144,1.254603,-0.289687
2,1,2018-05-01 02:00:00,0.07,286.26,-2.4345,-1.4461,NaN,NaN,NaN,NaN,NaN,99.990844,2.490908,-0.413370
3,1,2018-05-01 03:00:00,0.22,285.75,3.3659,-3.0605,2.6117,-2.3427,286.0,-1.148993,-2.274699,98.367235,0.997093,-1.415138
4,1,2018-05-01 04:00:00,0.39,284.78,3.7065,-6.2174,NaN,NaN,NaN,NaN,NaN,94.860604,0.689598,-0.961441
5,1,2018-05-01 05:00:00,0.41,284.46,3.8134,-5.4446,NaN,NaN,NaN,NaN,NaN,95.905879,0.290994,-0.294963


### Dealing with missing values

Missing values are interpolated linearly

In [14]:
def interpolate_nans(df):
    """ Interpolate missing values within each Wind farms """
    nwp_cols = [col for col in df.columns if col.startswith('NWP')]
    gb = df.groupby('WF')[nwp_cols]
    df.loc[:, nwp_cols] = gb.apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
    return df

In [16]:
df_interp = interpolate_nans(df_best)
df_interp.head()

,WF,Time,Production,NWP1_T,NWP1_U,NWP1_V,NWP2_U,NWP2_V,NWP3_T,NWP3_U,NWP3_V,NWP4_CLCT,NWP4_U,NWP4_V
ID,,,,,,,,,,,,,,
1,1,2018-05-01 01:00:00,0.02,286.44,-2.2485,-3.2578,2.6117,-2.3427,286.000000,-1.148993,-2.274699,82.543144,1.254603,-0.289687
2,1,2018-05-01 02:00:00,0.07,286.26,-2.4345,-1.4461,2.6117,-2.3427,286.000000,-1.148993,-2.274699,99.990844,2.490908,-0.413370
3,1,2018-05-01 03:00:00,0.22,285.75,3.3659,-3.0605,2.6117,-2.3427,286.000000,-1.148993,-2.274699,98.367235,0.997093,-1.415138
4,1,2018-05-01 04:00:00,0.39,284.78,3.7065,-6.2174,3.1783,-2.7838,285.666667,-0.518988,-2.721028,94.860604,0.689598,-0.961441
5,1,2018-05-01 05:00:00,0.41,284.46,3.8134,-5.4446,3.7449,-3.2249,285.333333,0.111017,-3.167357,95.905879,0.290994,-0.294963


### Feature engineering

Calculation of the Wind speed and the Wind direction

In [20]:
def append_features(df):
    """ Append wind speed and wind direction features in the dataframe"""
    nwp_cols = [col for col in df.columns if col.startswith('NWP')]
    for nwp_num in set([col.split('_')[0] for col in nwp_cols]):
        # Wind speed computation
        df.loc[:, nwp_num + '_WS'] = np.sqrt(df[nwp_num + '_U'] ** 2 + df[nwp_num + '_V'] ** 2)
        # Wind direction computation (angle)
        df.loc[:, nwp_num + '_WD'] = np.arctan2(df[nwp_num + '_U'], df[nwp_num + '_V'])  
    return df

In [21]:
df_aug = append_features(df_interp)
df_aug.loc[51:54]

,WF,Time,Production,NWP1_T,NWP1_U,NWP1_V,NWP2_U,NWP2_V,NWP3_T,NWP3_U,...,NWP4_U,NWP4_V,NWP1_WS,NWP1_WD,NWP2_WS,NWP2_WD,NWP4_WS,NWP4_WD,NWP3_WS,NWP3_WD
ID,,,,,,,,,,,,,,,,,,,,,
51,1,2018-05-03 03:00:00,5.39,288.350006,6.003520,-16.303010,4.850700,-12.651,286.299999,4.996607,...,4.450514,-6.487014,17.373267,2.788756,13.549062,2.775457,7.866920,2.540280,15.467656,2.812657
52,1,2018-05-03 04:00:00,5.11,288.120003,4.681039,-16.987960,4.440433,-13.064,286.199999,4.192226,...,4.369153,-7.600720,17.621093,2.872715,13.798027,2.813945,8.767009,2.619883,14.894384,2.856274
53,1,2018-05-03 05:00:00,5.03,287.710004,3.274157,-17.570962,4.030167,-13.477,286.100000,3.387845,...,3.888633,-8.621414,17.873410,2.957366,14.066690,2.851017,9.457815,2.717871,14.351678,2.903284
54,1,2018-05-03 06:00:00,7.18,287.519999,2.859262,-17.108018,3.619900,-13.890,286.000000,2.583464,...,3.100145,-9.882844,17.345306,2.975993,14.353946,2.886651,10.357679,2.837624,13.843133,2.953868


## Exploring data